In [15]:
import requests
import pandas as pd
from time import sleep
import numpy as np

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options




import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv("Data/df_hotels_22&21.csv", sep=',', encoding='UTF-8', index_col=0)
pd.options.display.max_columns = None
df.head(5)

,Hotel,Location,Country,Region,Company,Score,Rank,Rooms,Hotel_type,Theme,Year,Century,2021_Rank,Past_rank,Rank_Year
0,Rosewood Castiglion del Bosco,Montalcino,Italy,Europe,Massimo and Chiara Ferragamo,99.25,1,53,Small,Countryside,2000,21,0.0,0.0,2022
1,Grace Hotel,Santorini,Greece,Europe,Auberge Resorts Collection,99.22,2,20,Small,Coastal,2000,21,1.0,6.0,2022
2,Waldorf Astoria Maldives Ithaafushi,Ithaafushi Island,Maldives,Southeast Asia,Hilton,99.11,3,119,Small,Island,2019,21,1.0,80.0,2022
3,Pickering House Inn,Wolfeboro,United States,North America,Peter and Patty Cooke,98.95,4,10,Small,Boutique,1813,19,1.0,34.0,2022
4,One&Only Reethi Rah,North Malé Atoll,Maldives,Southeast Asia,Kerzner International,98.93,5,130,Small,Island,2005,21,0.0,0.0,2022


In [18]:
pd.set_option('display.max_rows', None)
df.head(1)

,Hotel,Location,Country,Region,Company,Score,Rank,Rooms,Hotel_type,Theme,Year,Century,2021_Rank,Past_rank,Rank_Year
0,Rosewood Castiglion del Bosco,Montalcino,Italy,Europe,Massimo and Chiara Ferragamo,99.25,1,53,Small,Countryside,2000,21,0.0,0.0,2022


In [22]:
print(df['Past_rank'].dtypes)

float64


Para evitar que me salgan errores en el scrapping de google voy a unir el hotel con la empresa

In [9]:
def add_columns(row):
    return f"hotel {row['Hotel']}"

In [14]:
df_hoteles_google = df[["Hotel"]]
df_hoteles_google["Hotel_buscador"] = df_hoteles_google.apply(join_columns, axis=1)
df_hoteles_google.head()

,Hotel,Hotel_buscador
0,Rosewood Castiglion del Bosco,hotel Rosewood Castiglion del Bosco
1,Grace Hotel,hotel Grace Hotel
2,Waldorf Astoria Maldives Ithaafushi,hotel Waldorf Astoria Maldives Ithaafushi
3,Pickering House Inn,hotel Pickering House Inn
4,One&Only Reethi Rah,hotel One&Only Reethi Rah


In [15]:
hotel_rev_google = df_hoteles_google["Hotel_buscador"].tolist()
print(hotel_rev_google)

['hotel Rosewood Castiglion del Bosco', 'hotel Grace Hotel', 'hotel Waldorf Astoria Maldives Ithaafushi', 'hotel Pickering House Inn', 'hotel One&Only Reethi Rah', 'hotel Royal Mansour Marrakech', 'hotel Capella Ubud', 'hotel The Lowell', 'hotel Hôtel Madame Rêve', 'hotel Rosewood Villa Magna', 'hotel The Oberoi New Delhi', 'hotel The Oberoi Udaivilas', 'hotel Mandapa, a Ritz-Carlton Reserve', 'hotel Wilderness Safaris Bisate Lodge', 'hotel Portrait Firenze', 'hotel Raffles Istanbul', 'hotel The Oberoi Marrakech', 'hotel Capella Hanoi', 'hotel White Elephant Palm Beach', 'hotel The Loutrel', 'hotel Monasterio', 'hotel Sani Asterias', 'hotel Shangri-La the Shard', 'hotel Coquillade Provence Resort & Spa', 'hotel Taj Palace', 'hotel Pendry Chicago', 'hotel Nayara Tented Camp', 'hotel Hotel Belmar', 'hotel Royal Champagne Hôtel & Spa', 'hotel Under Canvas Mount Rushmore', 'hotel Riggs Washington D.C.', 'hotel Waldorf Astoria Los Cabos Pedregal', 'hotel The Oberoi Vanyavilas Wildlife Resor

In [145]:
hotel_rev = df["Hotel"].unique().tolist()
print(hotel_rev)

['Rosewood Castiglion del Bosco', 'Grace Hotel', 'Waldorf Astoria Maldives Ithaafushi', 'Pickering House Inn', 'One&Only Reethi Rah', 'Royal Mansour Marrakech', 'Capella Ubud', 'The Lowell', 'Hôtel Madame Rêve', 'Rosewood Villa Magna', 'The Oberoi New Delhi', 'The Oberoi Udaivilas', 'Mandapa, a Ritz-Carlton Reserve', 'Wilderness Safaris Bisate Lodge', 'Portrait Firenze', 'Raffles Istanbul', 'The Oberoi Marrakech', 'Capella Hanoi', 'White Elephant Palm Beach', 'The Loutrel', 'Monasterio', 'Sani Asterias', 'Shangri-La the Shard', 'Coquillade Provence Resort & Spa', 'Taj Palace', 'Pendry Chicago', 'Nayara Tented Camp', 'Hotel Belmar', 'Royal Champagne Hôtel & Spa', 'Under Canvas Mount Rushmore', 'Riggs Washington D.C.', 'Waldorf Astoria Los Cabos Pedregal', 'The Oberoi Vanyavilas Wildlife Resort', 'Canaves Oia Epitome ', 'The Standard ', 'The Oberoi Amarvilas', 'Hotel Santa Caterina', "Raffles Grand Hotel d'Angkor", 'Nihi Sumba', 'La Réserve Paris', 'Hotel de la Ville', 'Il San Pietro di 

# Reviews tripadvisor

In [153]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.tripadvisor.com")

In [137]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [140]:
reviews_tripadvisor = {"Hotel": [],
                    "Trip_Rate": [], 
                    "Trip_Location": [], 
                    "Trip_Cleanliness": [], 
                    "Trip_Service": [], 
                    "Trip_Value": [], 
                    "Room_price": []}
    
for hotel in hotel_rev: 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(9)


        ## accedemos a la pagina web
    driver.get("https://www.tripadvisor.com")
    driver.implicitly_wait(14)

        # aceptamos las cookies                

    driver.find_element("css selector", "#onetrust-accept-btn-handler").click()
    sleep(7)

    driver.find_element("css selector", "#lithium-root > main > div.cBOoN > div.rGQXC._T.ivvQp > div > div > div:nth-child(1) > a > span.QLiHN.o.W > span").click()
    sleep(5)

    driver.find_element("css selector", "body > div.kaEuY > div > form > input.qjfqs._G.B-.z._J.Cj.R0").click()
    sleep(5)

    driver.find_element("css selector", "body > div.kaEuY > div > form > input.qjfqs._G.B-.z._J.Cj.R0").send_keys(f"{hotel}", Keys.TAB)
    sleep(6)

    driver.find_element("css selector", "#typeahead_results > a:nth-child(2) > div.EtzER > div.biGQs._P.fiohW.fOtGX").click()
    sleep(5)
        
    reviews_tripadvisor["Hotel"].append(f"{hotel}")

    try:
        reviews_tripadvisor["Trip_Rate"].append(float(driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.grdwI.P > span").text))
    except: 
        reviews_tripadvisor["Trip_Rate"].append(np.nan)

    try:
        reviews_tripadvisor["Trip_Location"].append(float(driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(1) > span.CzVMJ").text))
    except: 
        reviews_tripadvisor["Trip_Location"].append(np.nan)

    try:
        reviews_tripadvisor["Trip_Cleanliness"].append(float(driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(2) > span.CzVMJ").text))
    except: 
        reviews_tripadvisor["Trip_Cleanliness"].append(np.nan)

    try:
        reviews_tripadvisor["Trip_Service"].append(float(driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(3) > span.CzVMJ").text))
    except: 
        reviews_tripadvisor["Trip_Service"].append(np.nan)

    try:
        reviews_tripadvisor["Trip_Value"].append(float(driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(4) > span.CzVMJ").text))
    except: 
        reviews_tripadvisor["Trip_Value"].append(np.nan)

    driver.quit()

        #Trip_Rate = driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.grdwI.P > span").text
        #Trip_Location = driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(1) > span.CzVMJ").text
        #Trip_Cleanliness = driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(2) > span.CzVMJ").text
        #Trip_Service = driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(3) > span.CzVMJ").text
        #Trip_Value = driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(1) > div.SSDgd > div:nth-child(4) > span.CzVMJ").text
        #Room_price = driver.find_element("css selector", "#ABOUT_TAB > div > div:nth-child(2)").text

In [141]:
print(reviews_tripadvisor)

{'Hotel': ['Rosewood Castiglion del Bosco', 'Grace Hotel', 'Waldorf Astoria Maldives Ithaafushi', 'Pickering House Inn', 'One&Only Reethi Rah', 'Royal Mansour Marrakech', 'Capella Ubud', 'The Lowell', 'Hôtel Madame Rêve', 'Rosewood Villa Magna', 'The Oberoi New Delhi', 'The Oberoi Udaivilas', 'Mandapa, a Ritz-Carlton Reserve', 'Wilderness Safaris Bisate Lodge', 'Portrait Firenze', 'Raffles Istanbul', 'The Oberoi Marrakech', 'Capella Hanoi', 'White Elephant Palm Beach', 'The Loutrel', 'Monasterio', 'Sani Asterias', 'Shangri-La the Shard', 'Coquillade Provence Resort & Spa', 'Taj Palace', 'Pendry Chicago', 'Nayara Tented Camp', 'Hotel Belmar', 'Royal Champagne Hôtel & Spa', 'Under Canvas Mount Rushmore', 'Riggs Washington D.C.', 'Waldorf Astoria Los Cabos Pedregal', 'The Oberoi Vanyavilas Wildlife Resort', 'Canaves Oia Epitome ', 'The Standard ', 'The Oberoi Amarvilas', 'Hotel Santa Caterina', "Raffles Grand Hotel d'Angkor", 'Nihi Sumba', 'La Réserve Paris', 'Hotel de la Ville', 'Il San 

In [142]:
datos_trip = pd.DataFrame(reviews_tripadvisor, columns=["Hotel","Trip_Rate","Trip_Location","Trip_Cleanliness","Trip_Service","Trip_Value"])
datos_trip

,Hotel,Trip_Rate,Trip_Location,Trip_Cleanliness,Trip_Service,Trip_Value
0,Rosewood Castiglion del Bosco,5.0,4.7,4.9,4.8,4.4
1,Grace Hotel,5.0,4.9,4.9,4.8,4.4
2,Waldorf Astoria Maldives Ithaafushi,5.0,4.9,4.9,4.8,4.5
3,Pickering House Inn,5.0,5.0,4.8,4.9,4.4
4,One&Only Reethi Rah,5.0,4.8,4.9,4.8,4.2
5,Royal Mansour Marrakech,4.5,4.9,4.9,4.7,4.3
6,Capella Ubud,5.0,4.9,5.0,5.0,4.8
7,The Lowell,4.0,4.5,4.3,3.7,3.6
8,Hôtel Madame Rêve,4.5,4.6,4.6,4.6,4.3
9,Rosewood Villa Magna,4.5,4.7,4.8,4.7,4.2


# LIMPIEZA TRIP

relleno los dos hoteles vacios : Shangri-La Singapore	 & Il Sereno

In [155]:
datos_trip.loc[76, 'Trip_Rate'] = 4.5
datos_trip.loc[76, 'Trip_Location'] = 4.4
datos_trip.loc[76, 'Trip_Cleanliness'] = 4.7
datos_trip.loc[76, 'Trip_Service'] = 4.7
datos_trip.loc[76, 'Trip_Value'] = 4.2
datos_trip.loc[[76]]

,Hotel,Trip_Rate,Trip_Location,Trip_Cleanliness,Trip_Service,Trip_Value
76,Shangri-La Singapore,4.5,4.4,4.7,4.7,4.2


In [156]:
datos_trip.to_csv("Data/df_datos_trip.csv")
#hay solo 176 hoteles porque no necesito los que estan duplicados --> con el nombre del hotel es suficiente para linkearlos

# Reviews Google

In [16]:
    reviews_google = {"Hotel_buscador": [],
                    "Google_Rate": [], 
                    "Google_Habs": [], 
                    "Google_Location": [], 
                    "Google_Service": []}
    
    for hotel in hotel_rev_google: 
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.implicitly_wait(10)
            ## accedemos a la pagina web
        driver.get("https://www.google.com")
        driver.implicitly_wait(8)

        driver.find_element("css selector", "#L2AGLb > div").click()
        sleep(4)

        driver.find_element("css selector", "body > div.L3eUgb > div.o3j99.ikrT4e.om7nvf > form > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > input").send_keys(f"{hotel}", Keys.ENTER)
        sleep(10)
        
        reviews_google["Hotel_buscador"].append(hotel)
        
        try:
            reviews_google["Google_Rate"].append(float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[1]/div/div[2]/div[2]/div/div/span[1]').text.replace(",", ".")))
        except: 
            reviews_google["Google_Rate"].append(np.nan)

        try:
            reviews_google["Google_Habs"].append(float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/div/div/div/div/div/div/div/div[3]/div[1]/span[3]/span/span[1]').text.replace(",", ".")))
        except: 
            reviews_google["Google_Habs"].append(np.nan)

        try:
            reviews_google["Google_Location"].append(float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/div/div/div/div/div/div/div/div[3]/div[2]/span[3]/span/span[1]').text.replace(",", ".")))
        except: 
            reviews_google["Google_Location"].append(np.nan)

        try:
            reviews_google["Google_Service"].append(float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/div/div/div/div/div/div/div/div[3]/div[3]/span[3]/span/span[1]').text.replace(",", ".")))
        except: 
            reviews_google["Google_Service"].append(np.nan)

        driver.quit()
    
    #Google_Rate = float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[1]/div/div[2]/div[2]/div/div/span[1]').text.replace(",", "."))
    #Google_Habs = float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/div/div/div/div/div/div/div/div[3]/div[1]/span[3]/span/span[1]').text.replace(",", "."))
    #Google_Ubi =float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/div/div/div/div/div/div/div/div[3]/div[2]/span[3]/span/span[1]').text.replace(",", "."))
    #Google_Service =float(driver.find_element("xpath", '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/div/div/div/div/div/div/div/div[3]/div[3]/span[3]/span/span[1]').text.replace(",", "."))


[WDM] - Downloading: 100%|█████████████████| 8.61M/8.61M [00:00<00:00, 15.8MB/s]


In [21]:
datos_google = pd.DataFrame(reviews_google, columns=["Hotel","Google_Rate","Google_Habs","Google_Location","Google_Service"])
datos_google.head() 

,Hotel,Google_Rate,Google_Habs,Google_Location,Google_Service
0,NaN,4.8,4.3,4.1,4.6
1,NaN,4.4,NaN,NaN,NaN
2,NaN,4.7,4.6,4.6,4.6
3,NaN,4.9,NaN,NaN,NaN
4,NaN,4.7,4.1,4.6,4.2


In [1]:
datos_google = datos_google.assign(Hotel = df_hoteles_google["Hotel"])
datos_google.head(1)

NameError: name 'datos_google' is not defined

# Limpieza Scrapping Google

Busco los hoteles que no estan al completo rellenados y que contienen "NAN", para completarlos con loc()

In [23]:
datos_google.isnull()

,Hotel,Google_Rate,Google_Habs,Google_Location,Google_Service
0,True,False,False,False,False
1,True,False,True,True,True
2,True,False,False,False,False
3,True,False,True,True,True
4,True,False,False,False,False
5,True,False,False,False,False
6,True,False,False,False,False
7,True,False,True,True,True
8,True,False,False,False,False
9,True,False,True,True,True


In [34]:
datos_google.loc[1, 'Google_Rate'] = 4.6
datos_google.loc[1, 'Google_Habs'] = 4.8
datos_google.loc[1, 'Google_Location'] = 4.6
datos_google.loc[1, 'Google_Service'] = 4.5

datos_google.loc[[1]]

,Hotel,Google_Rate,Google_Habs,Google_Location,Google_Service
1,Grace Hotel,4.6,4.8,4.6,4.5


In [122]:
datos_google.loc[198, 'Google_Rate'] = 4.8
datos_google.loc[193, 'Google_Habs'] = 4.0
datos_google.loc[193, 'Google_Location'] = 4.3
datos_google.loc[193, 'Google_Service'] = 4.0

datos_google.loc[[96]]

,Hotel,Google_Rate,Google_Habs,Google_Location,Google_Service
96,Casa San Agustín,4.6,4.4,4.8,4.7


In [123]:
datos_google

,Hotel,Google_Rate,Google_Habs,Google_Location,Google_Service
0,Rosewood Castiglion del Bosco,4.8,4.3,4.1,4.6
1,Grace Hotel,4.6,4.8,4.6,4.5
2,Waldorf Astoria Maldives Ithaafushi,4.7,4.6,4.6,4.6
3,Pickering House Inn,4.6,4.8,4.6,4.5
4,One&Only Reethi Rah,4.9,4.1,4.6,4.2
5,Royal Mansour Marrakech,4.8,4.2,4.8,4.3
6,Capella Ubud,4.7,3.7,3.8,3.4
7,The Lowell,4.6,NaN,NaN,NaN
8,Hôtel Madame Rêve,4.4,4.2,4.8,3.9
9,Rosewood Villa Magna,4.5,NaN,NaN,NaN


In [143]:
datos_google.to_csv("Data/df_datos_google.csv")

In [10]:
datos_google = pd.read_csv("Data/df_datos_google.csv", sep=',', encoding='UTF-8', index_col=0)
datos_google.shape

(201, 5)

Me quito lo hoteles duplicados, tienen los mismos valores

In [12]:
datos_google_unicos = datos_google.drop_duplicates()
datos_google_unicos.shape

(178, 5)

In [13]:
datos_google_unicos.to_csv("Data/df_datos_google_unicos.csv")

In [14]:
datos_google_unicos.head(1)

,Hotel,Google_Rate,Google_Habs,Google_Location,Google_Service
0,Rosewood Castiglion del Bosco,4.8,4.3,4.1,4.6
